<a href="https://colab.research.google.com/github/gabrielfernandorey/EDVAI/blob/main/TP%20Final/Edvai_TP_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TRABAJO PRACTICO FINAL 
##### EDVAI

## Gabriel Rey 
##### Mayo 2023


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=9b072c3de63309a4beffec66a8062d1c38ebdad9d5b0ff3f41632478975bcfbb
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('test_pyspark').getOrCreate()

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, FloatType, IntegerType, DoubleType, DateType
import pyspark.sql.functions as F
from pyspark.sql.functions import sum, col, desc, asc, count, countDistinct, round, max, min, avg, when, lower
from pyspark.sql.functions import to_timestamp, date_format, to_date
from pyspark.sql.window import Window

from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, HasInputCols, HasOutputCols, Param, Params, TypeConverters
from pyspark import keyword_only
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml import Model
from pyspark.ml import Estimator

from datetime import datetime
import numpy as np
import pandas as pd

# Ejercicio 1

### Importamos datos

In [ ]:
!wget https://data-engineer-edvai.s3.amazonaws.com/2021-informe-ministerio.csv
!wget https://data-engineer-edvai.s3.amazonaws.com/202206-informe-ministerio.csv
!wget https://data-engineer-edvai.s3.amazonaws.com/aeropuertos_detalle.csv

--2023-05-17 21:17:49--  https://data-engineer-edvai.s3.amazonaws.com/2021-informe-ministerio.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 52.217.168.161, 52.216.163.3, 52.216.57.209, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|52.217.168.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32322556 (31M) [text/csv]
Saving to: ‘2021-informe-ministerio.csv’

2021-informe-minist 100%[===================>]  30.82M  26.9MB/s    in 1.1s    

2023-05-17 21:17:51 (26.9 MB/s) - ‘2021-informe-ministerio.csv’ saved [32322556/32322556]

--2023-05-17 21:17:51--  https://data-engineer-edvai.s3.amazonaws.com/202206-informe-ministerio.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 52.217.168.161, 52.216.163.3, 52.216.57.209, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|52.217.168.161|:4

In [ ]:
df_2021 = pd.read_csv("/content/2021-informe-ministerio.csv", sep=';')

In [ ]:
df_2022 = pd.read_csv("/content/202206-informe-ministerio.csv", sep=';')

<ipython-input-98-bcad1b5984a1>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv("/content/202206-informe-ministerio.csv", sep=';')


In [ ]:
df_aeropuertos = pd.read_csv("/content/aeropuertos_detalle.csv", sep=';')

### EDA

In [ ]:
df_2021

,Fecha,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,Calidad dato
0,01/01/2021,00:02,Vuelo Privado con Matrícula Nacional,Domestico,Despegue,PAR,ROS,0,PA-PA-28-181,0,DEFINITIVO
1,01/01/2021,00:24,Regular,Domestico,Aterrizaje,EZE,GRA,AEROLINEAS ARGENTINAS SA,BO-B737-8MB,70,DEFINITIVO
2,01/01/2021,00:26,Regular,Domestico,Aterrizaje,EZE,ECA,AEROLINEAS ARGENTINAS SA,BO-737-800,70,DEFINITIVO
3,01/01/2021,00:29,Regular,Domestico,Aterrizaje,EZE,SAL,AEROLINEAS ARGENTINAS SA,BO-B-737-76N,12,DEFINITIVO
4,01/01/2021,00:37,Regular,Domestico,Aterrizaje,EZE,TUC,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,26,DEFINITIVO
...,...,...,...,...,...,...,...,...,...,...,...
328131,31/12/2021,23:44,Regular,Domestico,Aterrizaje,JUJ,AER,FB LÍNEAS AÉREAS - FLYBONDI,BO-737-8Q8,96,DEFINITIVO
328132,31/12/2021,23:45,Regular,Domestico,Despegue,AER,CBA,FB LÍNEAS AÉREAS - FLYBONDI,BO-737-8Q8,88,DEFINITIVO
328133,31/12/2021,23:48,Regular,Domestico,Despegue,ROS,AER,AEROLINEAS ARGENTINAS SA,BO-737-800,10,DEFINITIVO
328134,31/12/2021,23:55,Regular,Internacional,Aterrizaje,EZE,SCEL,SKY AIRLINE S.A.,0,137,DEFINITIVO


In [ ]:
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328136 entries, 0 to 328135
Data columns (total 11 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   Fecha                              328136 non-null  object
 1   Hora UTC                           328136 non-null  object
 2   Clase de Vuelo (todos los vuelos)  328136 non-null  object
 3   Clasificación Vuelo                328136 non-null  object
 4   Tipo de Movimiento                 328136 non-null  object
 5   Aeropuerto                         328136 non-null  object
 6   Origen / Destino                   328136 non-null  object
 7   Aerolinea Nombre                   328136 non-null  object
 8   Aeronave                           328136 non-null  object
 9   Pasajeros                          328136 non-null  int64 
 10  Calidad dato                       328136 non-null  object
dtypes: int64(1), object(10)
memory usage: 27.5+ MB


In [ ]:
# Validamos si hay valores nulos en pasajeros
df_2021[df_2021['Pasajeros'].isna()] 


,Fecha,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,Calidad dato


In [ ]:
df_2022

,Fecha,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,Calidad dato
0,01/01/2022,00:01,Regular,Doméstico,Aterrizaje,AER,ECA,AEROLINEAS ARGENTINAS SA,BO-737-8SH,69,DEFINITIVO
1,01/01/2022,00:05,Regular,Doméstico,Aterrizaje,AER,SAL,AEROLINEAS ARGENTINAS SA,BO-B737-8,65,DEFINITIVO
2,01/01/2022,00:05,Regular,Doméstico,Despegue,IGU,AER,JETSMART AIRLINES S.A.,AIB-A320-232,41,DEFINITIVO
3,01/01/2022,00:09,Regular,Doméstico,Aterrizaje,AER,GAL,AEROLINEAS ARGENTINAS SA,BO-B737-81D,73,DEFINITIVO
4,01/01/2022,00:09,Regular,Internacional,Despegue,EZE,KDFW,AMERICAN AIRLINES INC.,0,261,DEFINITIVO
...,...,...,...,...,...,...,...,...,...,...,...
222922,30/06/2022,23:59,Regular,Doméstico,Despegue,DOZ,AER,AEROLINEAS ARGENTINAS SA,BO-B737-8SH,"83,5",PROVISORIO
222923,30/06/2022,16:26,Regular,Doméstico,Despegue,VIE,BAR,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,"33,5",PROVISORIO
222924,30/06/2022,13:08,Regular,Doméstico,Aterrizaje,VIE,AER,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,"45,5",PROVISORIO
222925,30/06/2022,01:33,Regular,Doméstico,Despegue,LAR,AER,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,"47,5",PROVISORIO


In [ ]:
# Validamos si hay valores nulos en pasajeros
df_2022[df_2022['Pasajeros'].isna()] 


,Fecha,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,Calidad dato


In [ ]:
df_aeropuertos

,local,oaci,iata,tipo,denominacion,coordenadas,latitud,longitud,elev,uom_elev,...,condicion,control,region,fir,uso,trafico,sna,concesionado,provincia,inhab
0,ACB,NaN,NaN,Aeródromo,CORONEL BOGADO/AGROSERVICIOS,"33°16'20""S 60°34'14""W",-60.570660,-33.272260,44.0,Metros,...,PRIVADO,NOCONTROL,RACE,SAEF,AEROAPP,Nacional,NO,NO,SANTA FÉ,NO
1,ACH,NaN,NaN,Aeródromo,GENERAL ACHA,"37°24' 6""S 64°36'49""W",-64.613510,-37.401640,277.0,Metros,...,PUBLICO,NOCONTROL,RACE,SAEF,CIVIL,Nacional,NO,NO,LA PAMPA,NO
2,ACM,NaN,NaN,Aeródromo,ARRECIFES/LA CURA MALAL,"34° 4'33""S 60° 8'30""W",-60.141700,-34.075740,37.0,Metros,...,PRIVADO,NOCONTROL,RACE,SAEF,CIVIL,Nacional,NO,NO,BUENOS AIRES,NO
3,ADO,SAWD,PUD,Aeródromo,PUERTO DESEADO,"47°44' 6""S 65°54'15""W",-65.904100,-47.735110,82.0,Metros,...,PUBLICO,AERADIO,RASU,SAVF,CIVIL,Nacional,NO,NO,SANTA CRUZ,NO
4,ADT,NaN,NaN,Aeródromo,BANDERA/AGROSERVICIOS DOÑA TERESA,"28°51'19""S 62°15'53""W",-62.264620,-28.855410,75.0,Metros,...,PRIVADO,NOCONTROL,RANO,SACF,AEROAPP,Nacional,NO,NO,SANTIAGO DEL ESTERO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,YOS,SAZH,OYO,Aeródromo,TRES ARROYOS,"38°23' 8""S 60°19'39""W",-60.327500,-38.385556,122.0,Metros,...,PUBLICO,NOCONTROL,RACE,SAEF,NaN,Nacional,NO,NO,BUENOS AIRES,NO
689,YPY,NaN,NaN,Aeródromo,YAPEYÚ,"29°21'32""S 56°47'45""W",-56.795730,-29.358770,53.0,Metros,...,PRIVADO,NOCONTROL,RANE,SARR,NaN,Nacional,NO,NO,CORRIENTES,NO
690,ZAP,SAHZ,APZ,Aeródromo,ZAPALA,"38°58'33""S 70° 6'48""W",-70.113464,-38.975886,1048.0,Metros,...,PUBLICO,NOCONTROL,RACE,SAEF,NaN,Nacional,NO,NO,NEUQUÉN,NO
691,ZLM,NaN,NaN,Aeródromo,BELL VILLE/LA ZULEMA,"32°29'23""S 62°40'17""W",-62.671389,-32.489722,129.0,Metros,...,PRIVADO,NOCONTROL,RANO,SACF,AEROAPP,Nacional,NO,NO,CÓRDOBA,NO


In [ ]:
df_aeropuertos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   local          693 non-null    object 
 1   oaci           124 non-null    object 
 2   iata           90 non-null     object 
 3   tipo           693 non-null    object 
 4   denominacion   693 non-null    object 
 5   coordenadas    693 non-null    object 
 6   latitud        693 non-null    float64
 7   longitud       693 non-null    float64
 8   elev           693 non-null    float64
 9   uom_elev       693 non-null    object 
 10  ref            683 non-null    object 
 11  distancia_ref  688 non-null    float64
 12  direccion_ref  679 non-null    object 
 13  condicion      693 non-null    object 
 14  control        693 non-null    object 
 15  region         693 non-null    object 
 16  fir            693 non-null    object 
 17  uso            470 non-null    object 
 18  trafico   

In [ ]:
# Validamos si hay valores nulos en distancia_ref
df_aeropuertos[df_aeropuertos['distancia_ref'].isna()] 

,local,oaci,iata,tipo,denominacion,coordenadas,latitud,longitud,elev,uom_elev,...,condicion,control,region,fir,uso,trafico,sna,concesionado,provincia,inhab
266,HET,NaN,NaN,Helipuerto,SANTIAGO DEL ESTERO/HELIPUERTO EDIFICIO TRIBUN...,"27°46'53""S 64°15'54""W",-64.265127,-27.781396,195.0,Metros,...,PRIVADO,NOCONTROL,RANO,SACF,NaN,Nacional,NO,NO,SANTIAGO DEL ESTERO,NO
299,HMG,NaN,NaN,Helipuerto,BUENOS AIRES / MADERO HARBOUR,"34°37'16""S 58°21'39""W",-58.360928,-34.620978,78.8,Metros,...,PRIVADO,NOCONTROL,RACE,SAEF,CIVIL,Nacional,NO,NO,CIUDAD AUTÓNOMA DE BUENOS AIRES,NO
307,HNH,NaN,NaN,Helipuerto,HELIPLATAFORMA NOBLE HOUSTON COLBERT,"53° 5'54""S 67°13'13""W",-67.220278,-53.098333,63.5,Metros,...,PRIVADO,NOCONTROL,RASU,SAVF,NaN,Nacional,NO,NO,TIERRA DEL FUEGO ANTÁRTIDA E ISLAS DEL ATLÁNTI...,NO
341,HSW,NaN,NaN,Helipuerto,HELIPLATAFORMA SWIBER PJW3000,"53° 4'35""S 67°57'50""W",-67.964004,-53.076495,36.0,Metros,...,PRIVADO,NOCONTROL,RASU,SAVF,NaN,Nacional,NO,NO,TIERRA DEL FUEGO ANTÁRTIDA E ISLAS DEL ATLÁNTI...,NO
439,MBI,SAWB,NaN,Aeródromo,VICECOMODORO MARAMBIO,"64°14'21""S 56°37'51""W",-56.630850,-64.239130,209.5,Metros,...,PUBLICO,CONTROL,RASU,SAVF,MIL,Nacional,NO,NO,TIERRA DEL FUEGO ANTÁRTIDA E ISLAS DEL ATLÁNTI...,NO


### SPARK



#### Transformaciones dataframes vuelos

In [ ]:
schema = StructType([
            StructField("Fecha", StringType(), True),
            StructField("horaUTC", StringType(), True),
            StructField("clase_de_vuelo", StringType(), True),
            StructField("clasificacion_de_vuelo", StringType(), True),
            StructField("tipo_de_movimiento", StringType(), True),
            StructField("aeropuerto", StringType(), True),
            StructField("origen_destino", StringType(), True),
            StructField("aerolinea_nombre", StringType(), True),
            StructField("aeronave", StringType(), True),
            StructField("pasajeros", IntegerType(), True)
])

In [ ]:
# Con esta alternativa los tipos de datos se importan de forma adecuada (siempre verificar!)
spk_2021 = spark.read.csv("/content/2021-informe-ministerio.csv", sep=';', schema=schema, header=True).withColumn("fecha", to_date("Fecha","dd/MM/yyyy"))

In [ ]:
spk_2021

DataFrame[fecha: date, horaUTC: string, clase_de_vuelo: string, clasificacion_de_vuelo: string, tipo_de_movimiento: string, aeropuerto: string, origen_destino: string, aerolinea_nombre: string, aeronave: string, pasajeros: int]

In [ ]:
spk_2021.show(5)

+----------+-------+--------------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|     fecha|horaUTC|      clase_de_vuelo|clasificacion_de_vuelo|tipo_de_movimiento|aeropuerto|origen_destino|    aerolinea_nombre|        aeronave|pasajeros|
+----------+-------+--------------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|2021-01-01|  00:02|Vuelo Privado con...|             Domestico|          Despegue|       PAR|           ROS|                   0|    PA-PA-28-181|        0|
|2021-01-01|  00:24|             Regular|             Domestico|        Aterrizaje|       EZE|           GRA|AEROLINEAS ARGENT...|     BO-B737-8MB|       70|
|2021-01-01|  00:26|             Regular|             Domestico|        Aterrizaje|       EZE|           ECA|AEROLINEAS ARGENT...|      BO-737-800|       70|
|2021-01-01|  00:29|             Regular|           

In [ ]:
spk_2021.tail(5)

[Row(fecha=datetime.date(2021, 12, 31), horaUTC='23:44', clase_de_vuelo='Regular', clasificacion_de_vuelo='Domestico', tipo_de_movimiento='Aterrizaje', aeropuerto='JUJ', origen_destino='AER', aerolinea_nombre='FB LÍNEAS AÉREAS - FLYBONDI', aeronave='BO-737-8Q8', pasajeros=96),
 Row(fecha=datetime.date(2021, 12, 31), horaUTC='23:45', clase_de_vuelo='Regular', clasificacion_de_vuelo='Domestico', tipo_de_movimiento='Despegue', aeropuerto='AER', origen_destino='CBA', aerolinea_nombre='FB LÍNEAS AÉREAS - FLYBONDI', aeronave='BO-737-8Q8', pasajeros=88),
 Row(fecha=datetime.date(2021, 12, 31), horaUTC='23:48', clase_de_vuelo='Regular', clasificacion_de_vuelo='Domestico', tipo_de_movimiento='Despegue', aeropuerto='ROS', origen_destino='AER', aerolinea_nombre='AEROLINEAS ARGENTINAS SA', aeronave='BO-737-800', pasajeros=10),
 Row(fecha=datetime.date(2021, 12, 31), horaUTC='23:55', clase_de_vuelo='Regular', clasificacion_de_vuelo='Internacional', tipo_de_movimiento='Aterrizaje', aeropuerto='EZE',

In [ ]:
# Validar si hay valores nulos y reemplazarlos por cero
spk_2021 = spk_2021.withColumn('pasajeros', when(col('pasajeros').isNull(), 0).otherwise(col('pasajeros')))


In [ ]:
spk_2022 = spark.read.csv("/content/202206-informe-ministerio.csv", sep=';', schema=schema, header=True).withColumn("fecha", to_date("Fecha","dd/MM/yyyy"))

In [ ]:
spk_2022.show(5)

+----------+-------+--------------+----------------------+------------------+----------+--------------+--------------------+------------+---------+
|     fecha|horaUTC|clase_de_vuelo|clasificacion_de_vuelo|tipo_de_movimiento|aeropuerto|origen_destino|    aerolinea_nombre|    aeronave|pasajeros|
+----------+-------+--------------+----------------------+------------------+----------+--------------+--------------------+------------+---------+
|2022-01-01|  00:01|       Regular|             Doméstico|        Aterrizaje|       AER|           ECA|AEROLINEAS ARGENT...|  BO-737-8SH|       69|
|2022-01-01|  00:05|       Regular|             Doméstico|        Aterrizaje|       AER|           SAL|AEROLINEAS ARGENT...|   BO-B737-8|       65|
|2022-01-01|  00:05|       Regular|             Doméstico|          Despegue|       IGU|           AER|JETSMART AIRLINES...|AIB-A320-232|       41|
|2022-01-01|  00:09|       Regular|             Doméstico|        Aterrizaje|       AER|           GAL|AEROLINEA

In [ ]:
spk_2022.tail(5)

[Row(fecha=datetime.date(2022, 6, 30), horaUTC='23:59', clase_de_vuelo='Regular', clasificacion_de_vuelo='Doméstico', tipo_de_movimiento='Despegue', aeropuerto='DOZ', origen_destino='AER', aerolinea_nombre='AEROLINEAS ARGENTINAS SA', aeronave='BO-B737-8SH', pasajeros=None),
 Row(fecha=datetime.date(2022, 6, 30), horaUTC='16:26', clase_de_vuelo='Regular', clasificacion_de_vuelo='Doméstico', tipo_de_movimiento='Despegue', aeropuerto='VIE', origen_destino='BAR', aerolinea_nombre='AEROLINEAS ARGENTINAS SA', aeronave='EMB-ERJ190100IGW', pasajeros=None),
 Row(fecha=datetime.date(2022, 6, 30), horaUTC='13:08', clase_de_vuelo='Regular', clasificacion_de_vuelo='Doméstico', tipo_de_movimiento='Aterrizaje', aeropuerto='VIE', origen_destino='AER', aerolinea_nombre='AEROLINEAS ARGENTINAS SA', aeronave='EMB-ERJ190100IGW', pasajeros=None),
 Row(fecha=datetime.date(2022, 6, 30), horaUTC='01:33', clase_de_vuelo='Regular', clasificacion_de_vuelo='Doméstico', tipo_de_movimiento='Despegue', aeropuerto='LA

In [ ]:
# Validar si hay valores nulos y reemplazarlos por cero
spk_2022 = spk_2022.withColumn('pasajeros', when(col('pasajeros').isNull(), 0).otherwise(col('pasajeros')))

#### Transformaciones dataframe aeropuerto

In [ ]:
schema2 = StructType([
            StructField("aeropuerto", StringType(), True),
            StructField("oac", StringType(), True),
            StructField("iata", StringType(), True),
            StructField("tipo", StringType(), True),
            StructField("denominacion", StringType(), True),
            StructField("coordenadas", StringType(), True),
            StructField("latitud", StringType(), True),
            StructField("longitud", StringType(), True),
            StructField("elev", FloatType(), True),
            StructField("uom_elev", StringType(), True),
            StructField("ref", StringType(), True),
            StructField("distancia_ref", FloatType(), True),
            StructField("condicion", StringType(), True),
            StructField("control", StringType(), True),
            StructField("region", StringType(), True),
            StructField("uso", StringType(), True),
            StructField("trafico", StringType(), True),
            StructField("sna", StringType(), True),
            StructField("concesionado", StringType(), True),
            StructField("provincia", StringType(), True)
])

In [ ]:
spk_aeropuertos = spark.read.csv("/content/aeropuertos_detalle.csv", sep=';', schema=schema2, header=True)

In [ ]:
spk_aeropuertos.show(5)

+----------+----+----+---------+--------------------+--------------------+------------+------------+-----+--------+--------------+-------------+---------+-------+---------+----+-------+-------+------------+---------+
|aeropuerto| oac|iata|     tipo|        denominacion|         coordenadas|     latitud|    longitud| elev|uom_elev|           ref|distancia_ref|condicion|control|   region| uso|trafico|    sna|concesionado|provincia|
+----------+----+----+---------+--------------------+--------------------+------------+------------+-----+--------+--------------+-------------+---------+-------+---------+----+-------+-------+------------+---------+
|       ACB|null|null|Aeródromo|CORONEL BOGADO/AG...|"33°16'20""S  60°...|-60.57066000|-33.27226000| 44.0|  Metros|Coronel Bogado|          6.0|       NE|PRIVADO|NOCONTROL|RACE|   SAEF|AEROAPP|    Nacional|       NO|
|       ACH|null|null|Aeródromo|        GENERAL ACHA|"37°24' 6""S  64°...|-64.61351000|-37.40164000|277.0|  Metros|  General Acha|  

### Union vuelos 2021-2022

In [ ]:
spk_vuelos = spk_2021.union(spk_2022)

In [ ]:
spk_vuelos.show(5)

+----------+-------+--------------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|     fecha|horaUTC|      clase_de_vuelo|clasificacion_de_vuelo|tipo_de_movimiento|aeropuerto|origen_destino|    aerolinea_nombre|        aeronave|pasajeros|
+----------+-------+--------------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|2021-01-01|  00:02|Vuelo Privado con...|             Domestico|          Despegue|       PAR|           ROS|                   0|    PA-PA-28-181|        0|
|2021-01-01|  00:24|             Regular|             Domestico|        Aterrizaje|       EZE|           GRA|AEROLINEAS ARGENT...|     BO-B737-8MB|       70|
|2021-01-01|  00:26|             Regular|             Domestico|        Aterrizaje|       EZE|           ECA|AEROLINEAS ARGENT...|      BO-737-800|       70|
|2021-01-01|  00:29|             Regular|           

Vuelos totales, nacionales e internacionales

In [ ]:
spk_vuelos.count()

551063

Vuelos domesticos

In [ ]:
spk_vuelos_nacionales = spk_vuelos.filter((spk_vuelos['clasificacion_de_vuelo'] == 'Doméstico') | (spk_vuelos['clasificacion_de_vuelo'] == 'Domestico') )

In [ ]:
spk_vuelos_nacionales.count()

483122

Determinar la cantidad de vuelos entre las fechas 01/12/2021 y 31/01/2022

In [ ]:
vuelos_nacionales_filtro = spk_vuelos_nacionales.filter((spk_vuelos_nacionales["fecha"] >= '2021-12-01') & (spk_vuelos_nacionales["fecha"] <= '2022-01-31'))

In [ ]:
vuelos_nacionales_filtro.count()

57984

Cantidad de pasajeros que viajaron en Aerolíneas Argentinas entre el 01/01/2021 y 30/06/2022

In [ ]:
pasajeros_aerolineasargentinas = spk_vuelos_nacionales.filter((spk_vuelos_nacionales["aerolinea_nombre"] == "AEROLINEAS ARGENTINAS SA") & (spk_vuelos_nacionales["fecha"] >= '2021-01-01') & (spk_vuelos_nacionales["fecha"] <= '2022-06-30'))

In [ ]:
total_pasajeros = pasajeros_aerolineasargentinas.agg(sum("pasajeros")).collect()[0][0]

In [ ]:
total_pasajeros

7484860

Mostrar fecha, hora, código aeropuerto salida, ciudad de salida, código de aeropuerto de arribo, ciudad de arribo, y cantidad de pasajeros de cada vuelo, entre el 01/01/2022 y el 30/06/2022 ordenados por fecha de manera descendiente. 


In [ ]:
spk_vuelos_nacionales.createOrReplaceTempView("vuelos_horarios")

In [ ]:
spk_aeropuertos.createOrReplaceTempView("aeropuertos")

In [ ]:
consulta_sql = """
SELECT vh.fecha, vh.horaUTC, vh.aeropuerto AS codigo_salida, a1.denominacion AS ciudad_salida, vh.origen_destino AS codigo_arribo, a2.denominacion AS ciudad_arribo, vh.pasajeros
FROM vuelos_horarios vh
JOIN aeropuertos a1 ON vh.aeropuerto = a1.aeropuerto
JOIN aeropuertos a2 ON vh.origen_destino = a2.aeropuerto
WHERE vh.fecha >= '2022-01-01'
ORDER BY vh.fecha DESC
"""

In [ ]:
vuelos_ordenados = spark.sql(consulta_sql)

In [ ]:
vuelos_ordenados.show(5)

+----------+-------+-------------+--------------------+-------------+--------------------+---------+
|     fecha|horaUTC|codigo_salida|       ciudad_salida|codigo_arribo|       ciudad_arribo|pasajeros|
+----------+-------+-------------+--------------------+-------------+--------------------+---------+
|2022-06-30|  00:02|          BAR|SAN CARLOS DE BAR...|          AER|BUENOS AIRES/AERO...|        0|
|2022-06-30|  00:12|          JUJ|JUJUY/GOBERNADOR ...|          EZE|EZEIZA/MINISTRO P...|        0|
|2022-06-30|  00:07|          FDO|        SAN FERNANDO|          CBA|CÓRDOBA/ING. AER....|        0|
|2022-06-30|  00:08|          ROS|ROSARIO/ISLAS MAL...|          FMA|CÓRDOBA/CAPITÁN D...|       48|
|2022-06-30|  00:09|          FDO|        SAN FERNANDO|          FDO|        SAN FERNANDO|        0|
+----------+-------+-------------+--------------------+-------------+--------------------+---------+
only showing top 5 rows



Cuáles son las 10 aerolíneas que más pasajeros llevaron entre el 01/01/2021 y el
30/06/2022 exceptuando aquellas aerolíneas que no tengan nombre?

In [ ]:
consulta_sql_2 = """
SELECT DISTINCT vh.aerolinea_nombre, SUM(vh.pasajeros) as total_pasajeros
FROM vuelos_horarios vh
WHERE vh.aerolinea_nombre !="0" and fecha BETWEEN '2021-01-01' AND '2022-06-30'
GROUP BY vh.aerolinea_nombre
ORDER BY total_pasajeros DESC, vh.aerolinea_nombre ASC
"""

In [ ]:
aerolineas_ordenado = spark.sql(consulta_sql_2)

In [ ]:
aerolineas_ordenado.show(10)

+--------------------+---------------+
|    aerolinea_nombre|total_pasajeros|
+--------------------+---------------+
|AEROLINEAS ARGENT...|        7484860|
|JETSMART AIRLINES...|        1511650|
|FB LÍNEAS AÉREAS ...|        1482473|
|   AMERICAN JET S.A.|          25789|
|            L.A.D.E.|          15074|
|       BAIRES FLY SA|           4960|
|                LADE|           3895|
|FUERZA AEREA ARGE...|           3855|
|FUERZA AEREA ARGE...|           3138|
|   FLYING AMERICA SA|           2839|
+--------------------+---------------+
only showing top 10 rows



Cuáles son las 10 aeronaves más utilizadas entre el 01/01/2021 y el 30/06/22 que despegaron desde la Ciudad autónoma de Buenos Aires o de Buenos Aires, exceptuando aquellas aeronaves que no cuentan con nombre?


# Ejercicio 2

In [6]:
!wget https://data-engineer-edvai.s3.amazonaws.com/CarRentalData.csv

--2023-05-22 18:24:28--  https://data-engineer-edvai.s3.amazonaws.com/CarRentalData.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 52.217.100.164, 3.5.16.146, 52.217.161.225, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|52.217.100.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 533157 (521K) [text/csv]
Saving to: ‘CarRentalData.csv’

CarRentalData.csv   100%[===================>] 520.66K  --.-KB/s    in 0.1s    

2023-05-22 18:24:28 (4.64 MB/s) - ‘CarRentalData.csv’ saved [533157/533157]



In [7]:
!wget -O /content/geo.csv 'https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/georef-united-states-of-america-state/exports/csv?lang=en&timezone=America%2FArgentina%2FBuenos_Aires&use_labels=true&delimiter=%3B'


--2023-05-22 18:24:28--  https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/georef-united-states-of-america-state/exports/csv?lang=en&timezone=America%2FArgentina%2FBuenos_Aires&use_labels=true&delimiter=%3B
Resolving public.opendatasoft.com (public.opendatasoft.com)... 34.248.20.69, 34.249.199.226
Connecting to public.opendatasoft.com (public.opendatasoft.com)|34.248.20.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘/content/geo.csv’

/content/geo.csv        [    <=>             ]   3.22M  5.12MB/s    in 0.6s    

2023-05-22 18:24:30 (5.12 MB/s) - ‘/content/geo.csv’ saved [3380726]



In [8]:
df_rental = pd.read_csv("/content/CarRentalData.csv", sep=',')

In [9]:
df_rental

,fuelType,rating,renterTripsTaken,reviewCount,location.city,location.country,location.latitude,location.longitude,location.state,owner.id,rate.daily,vehicle.make,vehicle.model,vehicle.type,vehicle.year
0,ELECTRIC,5.00,13,12,Seattle,US,47.449107,-122.308841,WA,12847615,135,Tesla,Model X,suv,2019
1,ELECTRIC,5.00,2,1,Tijeras,US,35.111060,-106.276551,NM,15621242,190,Tesla,Model X,suv,2018
2,HYBRID,4.92,28,24,Albuquerque,US,35.127163,-106.566681,NM,10199256,35,Toyota,Prius,car,2012
3,GASOLINE,5.00,21,20,Albuquerque,US,35.149726,-106.711425,NM,9365496,75,Ford,Mustang,car,2018
4,GASOLINE,5.00,3,1,Albuquerque,US,35.208659,-106.601008,NM,3553565,47,Chrysler,Sebring,car,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,GASOLINE,5.00,32,27,Honolulu,US,21.292950,-157.836856,HI,9794111,33,Chevrolet,Cruze,car,2017
5847,HYBRID,5.00,17,16,Aiea,US,21.375507,-157.914919,HI,2754690,49,Lexus,HS 250h,car,2010
5848,GASOLINE,4.94,18,17,Kailua,US,21.378719,-157.727816,HI,11313508,35,smart,fortwo,car,2013
5849,GASOLINE,NaN,1,0,Waipahu,US,21.376105,-158.020237,HI,4209883,77,GMC,Savana,van,2015


In [10]:
df_rental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5851 entries, 0 to 5850
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fuelType            5776 non-null   object 
 1   rating              5350 non-null   float64
 2   renterTripsTaken    5851 non-null   int64  
 3   reviewCount         5851 non-null   int64  
 4   location.city       5851 non-null   object 
 5   location.country    5851 non-null   object 
 6   location.latitude   5851 non-null   float64
 7   location.longitude  5851 non-null   float64
 8   location.state      5851 non-null   object 
 9   owner.id            5851 non-null   int64  
 10  rate.daily          5851 non-null   int64  
 11  vehicle.make        5851 non-null   object 
 12  vehicle.model       5851 non-null   object 
 13  vehicle.type        5851 non-null   object 
 14  vehicle.year        5851 non-null   int64  
dtypes: float64(3), int64(5), object(7)
memory usage: 685.8+

In [11]:
df_geo = pd.read_csv("/content/geo.csv", sep=';', usecols=[4, 7])

In [12]:
df_geo.head(5)

,Official Name State,United States Postal Service state abbreviation
0,Washington,WA
1,New Mexico,NM
2,Massachusetts,MA
3,Utah,UT
4,Arkansas,AR


### SPARK

#### Transformaciones 

In [28]:
schema = StructType([
            StructField("fuelType", StringType(), True),
            StructField("rating", FloatType(), True),
            StructField("renterTripsTaken", IntegerType(), True),
            StructField("reviewCount", IntegerType(), True),
            StructField("city", StringType(), True),
            StructField("aux1", StringType(), True),
            StructField("aux2", StringType(), True),
            StructField("aux3", StringType(), True),
            StructField("state_id", StringType(), True),
            StructField("owner_id", IntegerType(), True),
            StructField("rate_daily", IntegerType(), True),
            StructField("make", StringType(), True),
            StructField("model", StringType(), True),
            StructField("aux4", StringType(), True),
            StructField("year", IntegerType(), True)
])

In [29]:
spk_rental = spark.read.csv("/content/CarRentalData.csv", sep=',', schema=schema, header=True).withColumn("rating", round("rating").cast("integer")).withColumn("fuelType", lower(col("fuelType")))

In [30]:
spk_rental.show(5)

+--------+------+----------------+-----------+-----------+----+---------+-----------+--------+--------+----------+--------+-------+----+----+
|fuelType|rating|renterTripsTaken|reviewCount|       city|aux1|     aux2|       aux3|state_id|owner_id|rate_daily|    make|  model|aux4|year|
+--------+------+----------------+-----------+-----------+----+---------+-----------+--------+--------+----------+--------+-------+----+----+
|electric|     5|              13|         12|    Seattle|  US|47.449107|-122.308841|      WA|12847615|       135|   Tesla|Model X| suv|2019|
|electric|     5|               2|          1|    Tijeras|  US| 35.11106|-106.276551|      NM|15621242|       190|   Tesla|Model X| suv|2018|
|  hybrid|     5|              28|         24|Albuquerque|  US|35.127163|-106.566681|      NM|10199256|        35|  Toyota|  Prius| car|2012|
|gasoline|     5|              21|         20|Albuquerque|  US|35.149726|-106.711425|      NM| 9365496|        75|    Ford|Mustang| car|2018|
|gasol

In [31]:
spk_rental.printSchema()

root
 |-- fuelType: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- renterTripsTaken: integer (nullable = true)
 |-- reviewCount: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- aux1: string (nullable = true)
 |-- aux2: string (nullable = true)
 |-- aux3: string (nullable = true)
 |-- state_id: string (nullable = true)
 |-- owner_id: integer (nullable = true)
 |-- rate_daily: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- aux4: string (nullable = true)
 |-- year: integer (nullable = true)



In [32]:
spk_rental.count()

5851

In [33]:
spk_rental.createOrReplaceTempView("rental")

In [35]:
consulta_sql = """
SELECT * FROM rental
WHERE state_id !='TX' AND rating IS NOT NULL
"""
spk_rental_sinulos = spark.sql(consulta_sql)
spk_rental_sinulos.show(5)

+--------+------+----------------+-----------+-----------+----+---------+-----------+--------+--------+----------+--------+-------+----+----+
|fuelType|rating|renterTripsTaken|reviewCount|       city|aux1|     aux2|       aux3|state_id|owner_id|rate_daily|    make|  model|aux4|year|
+--------+------+----------------+-----------+-----------+----+---------+-----------+--------+--------+----------+--------+-------+----+----+
|electric|     5|              13|         12|    Seattle|  US|47.449107|-122.308841|      WA|12847615|       135|   Tesla|Model X| suv|2019|
|electric|     5|               2|          1|    Tijeras|  US| 35.11106|-106.276551|      NM|15621242|       190|   Tesla|Model X| suv|2018|
|  hybrid|     5|              28|         24|Albuquerque|  US|35.127163|-106.566681|      NM|10199256|        35|  Toyota|  Prius| car|2012|
|gasoline|     5|              21|         20|Albuquerque|  US|35.149726|-106.711425|      NM| 9365496|        75|    Ford|Mustang| car|2018|
|gasol

In [20]:
spk_rental_sinulos.count()

4905

#------------------------------------------------

In [36]:
schema_geo = StructType([
                StructField("Aux1", StringType(), True),
                StructField("Aux2", StringType(), True),
                StructField("Aux3", IntegerType(), True),
                StructField("Aux4", IntegerType(), True),
                StructField("state_name", StringType(), True),
                StructField("aux5", StringType(), True),
                StructField("aux6", StringType(), True),
                StructField("state_id", StringType(), True),
                StructField("aux7", StringType(), True),
                StructField("aux8", IntegerType(), True)
                
])

In [38]:
spk_geo = spark.read.csv("/content/geo.csv", sep=';', header=True, schema=schema_geo)

In [39]:
spk_geo.show(5)

+--------------------+--------------------+----+----+-------------+----+-----+--------+----+-------+
|                Aux1|                Aux2|Aux3|Aux4|   state_name|aux5| aux6|state_id|aux7|   aux8|
+--------------------+--------------------+----+----+-------------+----+-----+--------+----+-------+
|47.41161380972456...|"{""coordinates""...|2022|  53|   Washington| USA|state|      WA|null|1779804|
|34.42129899531876...|"{""coordinates""...|2022|  35|   New Mexico| USA|state|      NM|null| 897535|
|42.16009070994133...|"{""coordinates""...|2022|  25|Massachusetts| USA|state|      MA|null| 606926|
|39.32376674216668...|"{""coordinates""...|2022|  49|         Utah| USA|state|      UT|null|1455989|
|34.89991914658269...|"{""coordinates""...|2022|   5|     Arkansas| USA|state|      AR|null|  68085|
+--------------------+--------------------+----+----+-------------+----+-----+--------+----+-------+
only showing top 5 rows



In [40]:
spk_geo.printSchema()

root
 |-- Aux1: string (nullable = true)
 |-- Aux2: string (nullable = true)
 |-- Aux3: integer (nullable = true)
 |-- Aux4: integer (nullable = true)
 |-- state_name: string (nullable = true)
 |-- aux5: string (nullable = true)
 |-- aux6: string (nullable = true)
 |-- state_id: string (nullable = true)
 |-- aux7: string (nullable = true)
 |-- aux8: integer (nullable = true)



# ------------------------------------------------------

In [41]:
spk_rental_sinulos.createOrReplaceTempView("rental")

In [42]:
spk_geo.createOrReplaceTempView("geo")

In [43]:
consulta_sql_2 = """
SELECT r.fuelType, r.rating, r.renterTripsTaken, r.reviewCount, r.city, g.state_name, r.owner_id, r.rate_daily, r.make, r.model, r.year
FROM rental AS r
JOIN geo AS g ON g.state_id = r.state_id
ORDER BY r.rating DESC
"""
spk_rental = spark.sql(consulta_sql_2)
spk_rental.show(5)

+--------+------+----------------+-----------+--------+----------+--------+----------+---------+----------------+----+
|fuelType|rating|renterTripsTaken|reviewCount|    city|state_name|owner_id|rate_daily|     make|           model|year|
+--------+------+----------------+-----------+--------+----------+--------+----------+---------+----------------+----+
|  diesel|     5|               9|          8| Olympia|Washington| 6791294|        56|Chevrolet|Silverado 2500HD|2009|
|gasoline|     5|               8|          7|  Auburn|Washington|13165859|       128|  Porsche|         Cayenne|2014|
|gasoline|     5|              10|          7|Bellevue|Washington| 5103874|       150| Maserati|          Ghibli|2014|
|gasoline|     5|               2|          1|  Auburn|Washington|13165859|        60|     Ford|            Edge|2019|
|gasoline|     5|              49|         35|Puyallup|Washington| 3237150|        89|     Jeep|        Wrangler|2018|
+--------+------+----------------+-----------+--

#### Consultas DWH

a.	Cantidad de alquileres de autos, teniendo en cuenta sólo los vehículos ecológicos	(fuelType hibrido o eléctrico) y con un rating de al menos 4


In [46]:
spk_rental.createOrReplaceTempView("rental")

In [47]:
consulta_dwh_a = """
SELECT count(*) AS total
FROM rental
WHERE fuelType = 'electric' OR fuelType = 'hybrid' AND rating >= 4
"""
spk_dwh_a = spark.sql(consulta_dwh_a)
spk_dwh_a.show()

+-----+
|total|
+-----+
|  771|
+-----+



b. Los 5 estados con menor cantidad de alquileres (crear visualización)

In [53]:
consulta_dwh_b = """
SELECT state_name, COUNT(make, model) AS total
FROM rental
GROUP BY state_name
ORDER BY total ASC
LIMIT 5
"""
spk_dwh_b = spark.sql(consulta_dwh_b)
spk_dwh_b.show()

+-------------+-----+
|   state_name|total|
+-------------+-----+
|      Montana|    1|
|West Virginia|    3|
|New Hampshire|    3|
|     Delaware|    4|
|     Arkansas|    4|
+-------------+-----+



c.	Los 10 modelos (junto con su marca) de autos más rentados (crear visualización)

In [57]:
consulta_dwh_c = """
SELECT make, model, COUNT(make, model) AS total
FROM rental
GROUP BY make, model
ORDER BY total DESC
LIMIT 10
"""
spk_dwh_c = spark.sql(consulta_dwh_c)
spk_dwh_c.show()

+-------------+--------+-----+
|         make|   model|total|
+-------------+--------+-----+
|        Tesla| Model 3|  288|
|         Ford| Mustang|  136|
|        Tesla| Model S|  122|
|         Jeep|Wrangler|  108|
|        Tesla| Model X|  103|
|Mercedes-Benz| C-Class|   78|
|       Toyota| Corolla|   78|
|          BMW|3 Series|   76|
|    Chevrolet|Corvette|   68|
|    Chevrolet|  Camaro|   61|
+-------------+--------+-----+



d.	Por año, cuántos alquileres se hicieron, teniendo en cuenta desde 2010 a 2015

In [58]:
consulta_dwh_d = """
SELECT year, COUNT(year) AS total
FROM rental
WHERE year BETWEEN '2010' AND '2015'
GROUP BY year
ORDER BY total DESC
"""
spk_dwh_d = spark.sql(consulta_dwh_d)
spk_dwh_d.show()

+----+-----+
|year|total|
+----+-----+
|2015|  532|
|2014|  382|
|2013|  305|
|2012|  225|
|2011|  200|
|2010|  144|
+----+-----+



e.	Las 5 ciudades con más alquileres de vehículos ecológicos (fuelType hibrido o electrico)

In [59]:
consulta_dwh_e = """
SELECT state_name, COUNT(*) AS total
FROM rental
WHERE fuelType = 'electric' OR fuelType = 'hybrid'
GROUP BY state_name
ORDER BY total DESC
LIMIT 5
"""
spk_dwh_e = spark.sql(consulta_dwh_e)
spk_dwh_e.show()

+----------+-----+
|state_name|total|
+----------+-----+
|California|  201|
|   Florida|   77|
|    Nevada|   44|
|New Jersey|   41|
|   Arizona|   33|
+----------+-----+



f.	El promedio de reviews, segmentando por tipo de combustible

In [66]:
consulta_dwh_f = """
SELECT fuelType, round(AVG(reviewCount)) AS promedio
FROM rental
GROUP BY fuelType
ORDER BY promedio DESC
LIMIT 5
"""
spk_dwh_f = spark.sql(consulta_dwh_f)
spk_dwh_f.show()

+--------+--------+
|fuelType|promedio|
+--------+--------+
|  hybrid|    35.0|
|gasoline|    32.0|
|electric|    28.0|
|    null|    21.0|
|  diesel|    18.0|
+--------+--------+



In [69]:
consulta_dwh_g = """
SELECT count(*)
FROM rental
WHERE fuelType IS NULL
"""
spk_dwh_g = spark.sql(consulta_dwh_g)
spk_dwh_g.show()

+--------+
|count(1)|
+--------+
|      61|
+--------+

